## N-gram Model From Scratch

In this notebook, we'll implement a simple ngram language model using the text from Moby Dick.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
import random

from tqdm.notebook import tqdm

First, we'll read in the text and tokenize it into sentences.

In [ ]:
with open('moby_dick.txt') as fi:
    text = fi.read()
    
sentences = sent_tokenize(text)

First, let's build a simple unigram language model. This will only require us to keep up with the counts of each individual word.

For this, let's use a [defaultdict](https://docs.python.org/3/library/collections.html#defaultdict-objects) from the collections module. A behaves very much like a dictionary, but it allows us to specify what happens when we try to access a key which does not yet exist.

You might want to check out this page which talks about working with defaultdict objects: https://www.geeksforgeeks.org/defaultdict-in-python/

Create a defaultdict named `unigram_model` which contains the count of each word in the text. Do this by iterating through each sentence, tokenizing and then adding the count to unigram_model.

In [ ]:
# Your code here

In [ ]:
unigram_model = defaultdict(int)

for sentence in tqdm(sentences): 
    for word in word_tokenize(sentence):
        unigram_model[word] += 1

Now, write a function called `unigram_prob` which takes in a word as an argument and returns the probability (based on word counts) of seeing that word.

In [ ]:
# Your code here

In [ ]:
def unigram_prob(word):
    numerator = unigram_model[word]
    denominator = sum(unigram_model.values())
    return numerator / denominator

In [ ]:
unigram_prob('the')

In [ ]:
unigram_prob('whale')

Finally, create a function, `random_unigram` that randomly selects a word based on the probabilities above.

**Hint:** The [`random.choices` function](https://www.w3schools.com/python/ref_random_choices.asp) might be useful for this task, which allows you to randomly select an element from a list, with the chances of each word specified by a list of weights. 

In [ ]:
# Your Code Here

In [ ]:
def random_unigram():
    return random.choices(list(unigram_model.keys()), 
                          weights = unigram_model.values())[0]

Now, we can see what random text generated by your model looks like.

In [ ]:
sentence = []

for _ in range(50):
    sentence.append(random_unigram())

print(' '.join(sentence))

## Bigram Model

Now, let's build a bigram model. This will require us to count how many times each bigram appears. 

Let's do this by again utilizing a defaultdict, but this time we'll use a nested structure.

Create a defaultdict `bigram_model` where the keys are equal to the first word in a bigram and each value is a defaultdict containing counts of the number of times each word is the second part of a bigram starting with the first word.

For example, `bigram_model["white"]` would be a defaultdict counting the number of times each word shows up as the second part of a bigram which starts with "white". So `bigram_model["white"]["whale"]` gives the number of times that "white whale" shows up in our text and `bigram_model["accursed"]["whale"]` gives the number of times that "accursed whale" appears.

**Note:** To account for words that appear at the beginning of sentences, add a token `"<s>"` at the beginning of each sentence and one `"</s>"` at the end.

In [ ]:
# Your Code Here

In [ ]:
bigram_model = defaultdict(lambda: defaultdict(int))

for sentence in tqdm(sentences):
    sentence = ["<s>"] + word_tokenize(sentence) + ["</s>"]
    for first_word, second_word in zip(sentence[:-1], sentence[1:]):
        bigram_model[first_word][second_word] += 1

Now, write a function `random_bigram` which takes as an argument `seed` (which should default to `"<s>"`) and randomly selects a next word based on the bigram probabilities from your bigram model.

In [ ]:
# Your Code Here

In [ ]:
def random_bigram(seed = '<s>'):
    return random.choices(list(bigram_model[seed].keys()), 
                          weights = bigram_model[seed].values())[0]

Now, let's try it out and see how well it creates sentences.

Starting with a list containing `['<s>']`, use your `random_bigram` function to add tokens until you have added the `"</s>"` token. Then print out the resulting sentence.

In [ ]:
sentence = ['<s>']

# Fill in the code to generate a random sentence

print(' '.join(sentence))

In [ ]:
sentence = ['<s>']

while sentence[-1] != '</s>':
    sentence.append(random_bigram(sentence[-1]))
    
print(' '.join(sentence))

**Bonus** Extend your above work to trigram or even higher. What are the disadvantages of 3- or 4-gram models using the current text?

In [ ]:
trigram_model = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for sentence in tqdm(sentences):
    sentence = ["<s>", "<s>"] + word_tokenize(sentence) + ["</s>"]
    for first_word, second_word, third_word in zip(sentence[:-2], sentence[1:-1], sentence[2:]):
        trigram_model[first_word][second_word][third_word] += 1

In [ ]:
def random_trigram(seed = ['<s>', '<s>']):
    return random.choices(list(trigram_model[seed[0]][seed[1]].keys()), 
                          weights = trigram_model[seed[0]][seed[1]].values())[0]

In [ ]:
sentence = ['<s>', '<s>']

while sentence[-1] != '</s>':
    sentence.append(random_trigram(sentence[-2:]))
    
print(' '.join(sentence))

In [ ]:
quadgram_model = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))

for sentence in tqdm(sentences):
    sentence = ["<s>", "<s>", "<s>"] + word_tokenize(sentence) + ["</s>"]
    for first_word, second_word, third_word, fourth_word in zip(sentence[:-3], 
                                                   sentence[1:-2],
                                                   sentence[2:-1],
                                                  sentence[3:]):
        quadgram_model[first_word][second_word][third_word][fourth_word] += 1

In [ ]:
def random_quadgram(seed = ['<s>', '<s>', '<s>']):
    return random.choices(list(quadgram_model[seed[0]][seed[1]][seed[2]].keys()), 
                          weights = quadgram_model[seed[0]][seed[1]][seed[2]].values())[0]

In [ ]:
sentence = ['<s>', '<s>', '<s>']

while sentence[-1] != '</s>':
    sentence.append(random_quadgram(sentence[-3:]))
print(' '.join(sentence))

## Using NLTK Tools

In [ ]:
from nltk import word_tokenize, sent_tokenize
from nltk.lm import MLE, Laplace
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline

In [ ]:
ngram_size = 2

paddedLine = [list(pad_both_ends(word_tokenize(sentence), n = ngram_size)) for sentence in sent_tokenize(text)]

train, vocab = padded_everygram_pipeline(ngram_size, paddedLine)

lm = MLE(ngram_size)

lm.fit(train, vocab)

print(lm.counts)

In [ ]:
lm.perplexity(["Call", "me"])

In [ ]:
sentence = ['<s>']

while sentence[-1] != '</s>':
    sentence.append(lm.generate(text_seed = sentence))
print(' '.join(sentence))

In [ ]:
ngram_size = 3

paddedLine = [list(pad_both_ends(word_tokenize(sentence), n = ngram_size)) for sentence in sent_tokenize(moby)]

train, vocab = padded_everygram_pipeline(ngram_size, paddedLine)

lm = Laplace(ngram_size)

lm.fit(train, vocab)

print(lm.counts)

In [ ]:
sentence = ['<s>', '<s>']

while sentence[-1] != '</s>':
    sentence.append(lm.generate(text_seed = sentence))
print(' '.join(sentence))